In [1]:
!pip install google-generativeai
!pip install python-dotenv


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import google.generativeai as genai
from dotenv import load_dotenv

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv()
api_key_1 = os.getenv('GOOGLE_API_KEY_1')
api_key_2 = os.getenv('GOOGLE_API_KEY_2')
api_key_3 = os.getenv('GOOGLE_API_KEY_3')
api_key_4 = os.getenv('GOOGLE_API_KEY_4')
api_key_5 = os.getenv('GOOGLE_API_KEY_5')
api_key_6 = os.getenv('GOOGLE_API_KEY_6')

api_keys = [
    api_key_1,
    api_key_2,
    api_key_3,
    api_key_4,
    api_key_5,
    api_key_6
]

In [4]:
filenames = [
    "group_1.txt",
    "group_2.txt",
    "group_3.txt",
    "group_4.txt",
    "group_5.txt",
    "group_6.txt",
    "group_7.txt",
    "group_8.txt",
    "group_9.txt",
    "group_10.txt",
    "group_11.txt",
    "group_12.txt",
]

In [5]:
file_contents = []

In [6]:
for filename in filenames:
    file = open(f"./data/{filename}", "r")
    file_content = file.read()
    file.close()
    file_contents.append(file_content)

In [7]:
question = "Kas ma võin autoga sõites alkoholi tarbida?"

In [8]:
system_prompt = "Sa oled seaduste abiline. Kui sulle antud seaduses on küsimusele vastust, siis vasta sellele. Kui vastus puudub, siis vasta '0'"

In [9]:
response_format = """kastuta vastamisel järgmist formaati:
<seaduse nimi>

<sinu vastus>"""

In [10]:
responses = []

In [ ]:
group_index = 0
group_count = len(api_keys) * 2

for api_key in api_keys:
    genai.configure(api_key=api_key)

    print(f"progress: {group_index + 1}/{group_count}")
    file_content = file_contents[group_index]
    model = genai.GenerativeModel("gemini-1.5-flash")
    response_text = model.generate_content([
        system_prompt,
        response_format,
        file_content,
        "Küsimus on järgmine:",
        question
    ]).text
    responses.append(response_text)

    group_index += 1

    print(f"progress: {group_index + 1}/{group_count}")
    file_content = file_contents[group_index]
    model = genai.GenerativeModel("gemini-1.5-flash-8b")
    response_text = model.generate_content([
        system_prompt,
        response_format,
        file_content,
        "Küsimus on järgmine:",
        question
    ]).text
    responses.append(response_text)

    group_index += 1

In [22]:
group_index = 0
group_count = len(api_keys) * 2

thread_local = threading.local()
responses_lock = threading.Lock()


def get_session():
    if not hasattr(thread_local, "session"):
        thread_local.session = requests.Session()
    return thread_local.session

def get_response(data):
    api_key, file_content, group_index, group_count, system_prompt, response_format, question = data
    
    genai.configure(api_key=api_key)

    
    try:

        # Use generative model
        model = genai.GenerativeModel("gemini-1.5-flash")
        response_text = model.generate_content([
            system_prompt,
            response_format,
            file_content,
            "Küsimus on järgmine:",
            question
        ]).text

        # Safely append to the shared responses list
        with responses_lock:
            responses.append((group_index, response_text))

        print(f"Progress: {group_index + 1}/{group_count}")
    except Exception as e:
        print(f"Error processing group {group_index}: {e}")

        
def get_all_responses(api_keys, file_contents, system_prompt, response_format, question):
    group_count = len(api_keys) * len(file_contents)
    tasks = [
        ( api_keys[i % len(api_keys)], file_contents[i], i, group_count, system_prompt, response_format, question)
        for i in range(len(file_contents))
    ]

    with concurrent.futures.ThreadPoolExecutor(max_workers=12) as executor:
        executor.map(get_response, tasks)

if __name__ == "__main__":
    
    api_keys = api_keys * 2

    start_time = time.time()
    
    get_all_responses(api_keys, file_contents, system_prompt, response_format, question)
    
    duration = time.time() - start_time

    responses.sort(key=lambda x: x[0])
    for idx, response_text in responses:
        print(f"Response {idx + 1}: {response_text}")

    print(f"I did it in {duration} seconds")


Error processing group 9: 429 Resource has been exhausted (e.g. check quota).
Error processing group 2: 429 Resource has been exhausted (e.g. check quota).
Error processing group 3: 429 Resource has been exhausted (e.g. check quota).
Error processing group 1: 429 Resource has been exhausted (e.g. check quota).
Error processing group 5: 429 Resource has been exhausted (e.g. check quota).
Progress: 12/288
Error processing group 7: 429 Resource has been exhausted (e.g. check quota).
Progress: 1/288
Progress: 9/288Progress: 7/288

Progress: 11/288
Progress: 5/288
Response 1: Võlaõigusseadus

0

Response 5: Sotsiaalhoolekande seadus

0

Response 7: Autoveoseadus

Ei, sa ei või autoga sõites alkoholi tarbida.  Autoveoseaduse § 15 lõikes 1 punkt 4 on sätestatud, et autojuht ei tohi viibida mootorsõidukis joobeseisundis.

Response 9: Tubakaseadus

0

Response 11: Reklaamiseadus

0

Response 12: 0

I did it in 57.627875089645386 seconds


In [23]:
for response in responses:
    print(response)

(0, 'Võlaõigusseadus\n\n0\n')
(4, 'Sotsiaalhoolekande seadus\n\n0\n')
(6, 'Autoveoseadus\n\nEi, sa ei või autoga sõites alkoholi tarbida.  Autoveoseaduse § 15 lõikes 1 punkt 4 on sätestatud, et autojuht ei tohi viibida mootorsõidukis joobeseisundis.\n')
(8, 'Tubakaseadus\n\n0\n')
(10, 'Reklaamiseadus\n\n0\n')
(11, '0\n')


In [24]:
genai.configure(api_key=api_keys[0])
model = genai.GenerativeModel("gemini-1.5-pro")

prompt = (
    ["Sulle antakse mitme erineva mudeli vastused erinevatest seadustest, tsiteeri mulle mudeli vastuseid, mis pole '0'"] +
    [response_format] +
    responses + 
    ["Küsimus on järgnev:"] +
    [question]
)

best_answer = model.generate_content(prompt)

TypeError: Could not create `Blob`, expected `Blob`, `dict` or an `Image` type(`PIL.Image.Image` or `IPython.display.Image`).
Got a: <class 'tuple'>
Value: (0, 'Võlaõigusseadus\n\n0\n')

In [45]:
print(best_answer.text)

**Karistusseadustik**

Ei. Karistusseadustiku § 424 sätestab, et mootorsõiduki juhtimise eest joobeseisundis karistatakse rahalise karistuse või kuni kolmeaastase vangistusega.

**Autoveoseadus**

Ei, see on keelatud. Autoveoseaduse § 97 lõige 1 sätestab, et autojuht ei tohi olla alkoholi tarvitanud.



In [20]:
%%html
<html>
<head>
  <link rel="stylesheet" href="styles.css">
</head>
<body>
<div class="header">
    <h1>juur<span>.ai</span></h1>
</div>

<div class="input-footer">
    <textarea type="text" placeholder="Ask anything" class="input-text glassmorphism"> </textarea>
</div>
</body>
</html>